# Torus one-point functions of the $O(n)$ model

In [10]:
Pkg.activate("."); # activate the parent environment
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BenchmarkTools
println("Number of threads: $(Threads.nthreads())")

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`


Number of threads: 4


In [ ]:
const Sig = Channels{Rational}

c = CC(β=-big"0.8"-big"0.1"*im)
ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r*s % 1//2 == 0]
diag_field = Field(c, r=0, s=big"0.4"+big"0.1"*im)
fields = vcat([Field(c, r = r, s = s) for (r, s) in ndiag_indices], diag_field)
Δmax = 10
setprecision(BigFloat, 8, base=10)
co = Correlation(Field(c, r=1, s=0), Δmax)

function torus_channelspectrum(co, fields; parity)
    parity != 0 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    ChannelSpectrum(co, fields, V -> IBlock(co, V, parity=1))
end

function torus_spectra(fields, signature)
    Vs = @channels filter(V -> V.r >= signature[chan], fields)
    @channels torus_channelspectrum(co, Vs[chan], parity=1)
end

torus_spectra (generic function with 1 method)

In [13]:
sig = Sig(1, 1, 1)
specs = torus_spectra(fields, sig);

sys = BootstrapSystem(specs)
evaluate_blocks!(sys)
compute_linear_system!(sys);
solve!(sys)
show(stdout, sys.str_cst, rmax=2)

 Channel s
┌──────────────┬────────────────────┬────────────────┐
│ Field        │ Structure constant │ Relative error │
├──────────────┼────────────────────┼────────────────┤
│       (1, 0) │          1.0+0.0im │            0.0 │
│    (1, 1//2) │ 0.588295+1.94315im │        3.11977 │
│    (3//2, 0) │ -6.17451-9.46857im │       0.564183 │
│ (3//2, 1//3) │  9.62008-9.63231im │        2.35228 │
│ (3//2, 2//3) │  5.11095-35.3588im │        1.94307 │
│       (1, 1) │  47.4123+5.51682im │        2.32943 │
│       (2, 0) │ -295.115+2.78228im │        1.81841 │
│    (2, 1//4) │  62.8311-122.989im │         1.3369 │
│    (2, 1//2) │ -69.6903+89.6207im │        1.25148 │
│    (3//2, 1) │ 4.65511+0.670888im │        3.23408 │
│    (2, 3//4) │   177.388+516.94im │        2.29432 │
│       (2, 1) │  -1434.53+2418.9im │        1.71274 │
└──────────────┴────────────────────┴────────────────┘
 Channel t
┌──────────────┬────────────────────┬────────────────┐
│ Field        │ Structure constant │ Relat

In [76]:
sys.positions

23-element Vector{Complex{BigFloat}}:
  0.1808480527 + 0.5740926042im
  0.2284243442 + 0.5458371565im
  0.1552194115 + 0.5889387056im
  0.1637292057 + 0.4784067497im
  0.1875439044 + 0.5335608795im
  0.1826739181 + 0.6845091358im
   0.145901341 + 0.4842026532im
  0.2040876448 + 0.6092510223im
   0.109705504 + 0.5661882311im
  0.2146782186 + 0.5893488452im
               ⋮
 0.07138148695 + 0.5502460822im
   0.108753223 + 0.5131443366im
 0.09950471483 + 0.5295872837im
  0.1579050273 + 0.4943104833im
  0.2060903981 + 0.5270898044im
  0.1280253567 + 0.5126438215im
  0.1255315691 + 0.4873910993im
  0.1704721227 + 0.6282873303im
  0.0677560335 + 0.5117262751im

In [ ]:
function prefactor_f(τ, co_s, chan, lr)
    x_cache = PosCache(xfromτ(τ), co_s[lr], chan)
    return inv(
        etaDedekind(τ) * x_cache.prefactor
    )
end

nonchiral_prefactor_f(τ, co_s, chan) = prefactor_f(τ, co_s, chan, :left) * 
    prefactor_f(-conj(τ), co_s, chan, :right)